## Setup

In [69]:
import os
import base64
from requests import post , get
import json
import pandas as pd
from dotenv import load_dotenv
import numpy as np
from functools import cache
import time

In [70]:
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
# leaving these public because there's no need to make env-vars
client_id = "e19f868c40394ceeb90e778daf20f9c7"
client_secret = "6be8e17dfbe347f0976fbdf00f500bc9"

In [71]:

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data={
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }

    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    return json_result['access_token']

def get_auth_header(token):
    return {
        "Authorization": "Bearer " + token,
    }


In [72]:
token = get_token()
header = get_auth_header(token)
print(token)


BQCJsma7KkygM5nJC1l8XNLfTQm_zDzSoRyqynuhjSZhg-OUXhbtWiOW72e1eEQXqgtjdp_XhsDsM_3d-r1rmhBbxTKCdPqHgV9qfWPI_kJ3T2gEtBY


## ALBUM

In [73]:
def get_album(album_id, market=None):
    """
    Get Spotify catalog information for a single album.

    Parameters:
    - album_id (str): The Spotify ID of the album.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.

    Returns:
    - dict: JSON response containing the album's information.
    """
    url = f"https://api.spotify.com/v1/albums/{album_id}"
    headers = header
    params = {}

    if market:
        params["market"] = market

    response = get(url, headers=headers, params=params)
    return response.json()

def get_several_albums(album_ids, market=None):
    """
    Get Spotify catalog information for multiple albums identified by their Spotify IDs.

    Parameters:
    - album_ids (list): A list of Spotify IDs for the albums. Maximum: 20 IDs.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.

    Returns:
    - dict: JSON response containing the albums' information.
    """
    url = "https://api.spotify.com/v1/albums"
    headers = header
    params = {
        "ids": ",".join(album_ids)
    }

    if market:
        params["market"] = market

    response = get(url, headers=headers, params=params)
    return response.json()

def get_album_tracks(album_id, market=None, limit=20, offset=0):
    """
    Get Spotify catalog information about an album’s tracks.

    Parameters:
    - album_id (str): The Spotify ID of the album.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.
    - limit (int, optional): The maximum number of items to return. Default is 20. Range is 1 to 50.
    - offset (int, optional): The index of the first item to return. Default is 0.

    Returns:
    - dict: JSON response containing the album's tracks information.
    """
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    if market:
        params["market"] = market

    response = get(url, headers=headers, params=params)
    return response.json()

## ARTIST

In [74]:
def get_artist(artist_id):
    """
    Get Spotify catalog information for a single artist.

    Parameters:
    - artist_id (str): The Spotify ID of the artist.

    Returns:
    - dict: JSON response containing the artist's information.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = header

    response = get(url, headers=headers)
    return response.json()

def get_several_artists(artist_ids):
    """
    Get Spotify catalog information for several artists based on their Spotify IDs.

    Parameters:
    - artist_ids (list): A list of Spotify IDs for the artists. Maximum: 50 IDs.

    Returns:
    - dict: JSON response containing the artists' information.
    """
    url = "https://api.spotify.com/v1/artists"
    headers = header
    params = {
        "ids": ",".join(artist_ids)
    }

    response = get(url, headers=headers, params=params)
    return response.json()

def get_artist_albums(artist_id, include_groups=None, market=None, limit=20, offset=0):
    """
    Get Spotify catalog information about an artist's albums.

    Parameters:
    - artist_id (str): The Spotify ID of the artist.
    - include_groups (str, optional): A comma-separated list of keywords to filter the response. Valid values are "album", "single", "appears_on", "compilation".
    - market (str, optional): An ISO 3166-1 alpha-2 country code.
    - limit (int, optional): The maximum number of items to return. Default is 20. Range is 1 to 50.
    - offset (int, optional): The index of the first item to return. Default is 0.

    Returns:
    - dict: JSON response containing the artist's albums information.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    if include_groups:
        params["include_groups"] = include_groups
    if market:
        params["market"] = market

    response = get(url, headers=headers, params=params)
    return response.json()

def get_artist_top_tracks(artist_id, market=None):
    """
    Get Spotify catalog information about an artist's top tracks by country.

    Parameters:
    - artist_id (str): The Spotify ID of the artist.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.

    Returns:
    - dict: JSON response containing the artist's top tracks information.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
    headers = header
    params = {}

    if market:
        params["market"] = market

    response = get(url, headers=headers, params=params)
    return response.json()

def get_related_artists(artist_id):
    """
    Get Spotify catalog information about artists similar to a given artist.

    Parameters:
    - artist_id (str): The Spotify ID of the artist.

    Returns:
    - dict: JSON response containing the related artists' information.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/related-artists"
    headers = header

    response = get(url, headers=headers)
    return response.json()

## CATEGORIES

In [75]:
def get_categories(locale=None, limit=20, offset=0):
    """
    Get a list of categories used to tag items in Spotify (on, for example, the Spotify player’s “Browse” tab).

    Parameters:
    - locale (str, optional): The desired language, consisting of an ISO 639-1 language code and an ISO 3166-1 alpha-2 country code, joined by an underscore.
    - limit (int, optional): The maximum number of items to return. Default is 20. Minimum is 1. Maximum is 50.
    - offset (int, optional): The index of the first item to return. Default is 0.

    Returns:
    - dict: JSON response containing the categories information.
    """
    url = "https://api.spotify.com/v1/browse/categories"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    if locale:
        params["locale"] = locale

    response = get(url, headers=headers, params=params)
    return response.json()

def get_single_browse_category(category_id, locale=None):
    """
    Get a single category used to tag items in Spotify.

    Parameters:
    - category_id (str): The Spotify category ID for the category.
    - locale (str, optional): The desired language, consisting of an ISO 639-1 language code and an ISO 3166-1 alpha-2 country code, joined by an underscore.

    Returns:
    - dict: JSON response containing the category's information.
    """
    url = f"https://api.spotify.com/v1/browse/categories/{category_id}"
    headers = header
    params = {}

    if locale:
        params["locale"] = locale

    response = get(url, headers=headers, params=params)
    return response.json()

## PLAYLIST

In [76]:
def get_playlist(playlist_id, market=None, fields=None, additional_types=None):
    """
    Get a playlist owned by a Spotify user.

    Parameters:
    - playlist_id (str): The Spotify ID of the playlist.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.
    - fields (str, optional): Filters for the query.
    - additional_types (str, optional): A comma-separated list of item types that your client supports besides the default track type.

    Returns:
    - dict: JSON response containing the playlist's information.
    """
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = header
    params = {}

    if market:
        params["market"] = market
    if fields:
        params["fields"] = fields
    if additional_types:
        params["additional_types"] = additional_types

    response = get(url, headers=headers, params=params)
    return response.json()

def get_playlist_items(playlist_id, market=None, fields=None, limit=20, offset=0, additional_types=None):
    """
    Get full details of the items of a playlist owned by a Spotify user.

    Parameters:
    - playlist_id (str): The Spotify ID of the playlist.
    - market (str, optional): An ISO 3166-1 alpha-2 country code.
    - fields (str, optional): Filters for the query.
    - limit (int, optional): The maximum number of items to return. Default is 20. Range is 1 to 50.
    - offset (int, optional): The index of the first item to return. Default is 0.
    - additional_types (str, optional): A comma-separated list of item types that your client supports besides the default track type.

    Returns:
    - dict: JSON response containing the playlist items' information.
    """
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    if market:
        params["market"] = market
    if fields:
        params["fields"] = fields
    if additional_types:
        params["additional_types"] = additional_types

    response = get(url, headers=headers, params=params)
    return response.json()

def get_featured_playlists(locale=None, limit=20, offset=0):
    """
    Get a list of Spotify featured playlists.

    Parameters:
    - locale (str, optional): The desired language, consisting of an ISO 639-1 language code and an ISO 3166-1 alpha-2 country code, joined by an underscore.
    - limit (int, optional): The maximum number of items to return. Default is 20. Minimum is 1. Maximum is 50.
    - offset (int, optional): The index of the first item to return. Default is 0.

    Returns:
    - dict: JSON response containing the featured playlists information.
    """
    url = "https://api.spotify.com/v1/browse/featured-playlists"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    if locale:
        params["locale"] = locale

    response = get(url, headers=headers, params=params)
    return response.json()

def get_category_playlists(category_id, limit=20, offset=0):
    """
    Get a list of Spotify playlists tagged with a particular category.

    Parameters:
    - category_id (str): The Spotify category ID for the category.
    - limit (int, optional): The maximum number of items to return. Default is 20. Minimum is 1. Maximum is 50.
    - offset (int, optional): The index of the first item to return. Default is 0.

    Returns:
    - dict: JSON response containing the playlists information.
    """
    url = f"https://api.spotify.com/v1/browse/categories/{category_id}/playlists"
    headers = header
    params = {
        "limit": limit,
        "offset": offset
    }

    response = get(url, headers=headers, params=params)
    return response.json()

## SEARCH

In [77]:
def search_spotify_items(q, types, market=None, limit=20, offset=0, include_external=None):
    """
    Search for Spotify catalog information about albums, artists, playlists, tracks, shows, episodes, or audiobooks.

    Parameters:
    - q (str): The search query.
    - types (list): A list of item types to search across. Allowed values: "album", "artist", "playlist", "track", "show", "episode", "audiobook".
    - market (str, optional): An ISO 3166-1 alpha-2 country code.
    - limit (int, optional): The maximum number of results to return in each item type. Default is 20. Range is 0 - 50.
    - offset (int, optional): The index of the first result to return. Default is 0. Range is 0 - 1000.
    - include_external (str, optional): If "audio", includes externally hosted audio content as playable.

    Returns:
    - dict: JSON response containing the search results.
    """
    base_url = "https://api.spotify.com/v1/search"
    headers = header
    params = {
        "q": q,
        "type": ",".join(types),
        "limit": limit,
        "offset": offset
    }

    if market:
        params["market"] = market
    if include_external:
        params["include_external"] = include_external

    response = get(base_url, headers=headers, params=params)
    return response.json()

# Trabalho

### DEFININDO GENEROS

In [78]:
categories = {
    "names": [],
    "ids": []
}

categories_data = get_categories(locale="BR", limit=50, offset=0)['categories']['items']
names = np.array([category['name'] for category in categories_data])
ids = np.array([category['id'] for category in categories_data])

# Efficiently assign lists without a loop
categories["names"] = names.tolist()
categories["ids"] = ids.tolist()
categories

{'names': ['Conçu spécialement pour vous',
  'Dernières sorties',
  'Marília\xa0Mendonça',
  'Brésil',
  'Sertanejo',
  'Pop',
  'Classements',
  'Funk brésilien',
  'En voiture',
  'Hip-Hop',
  'Samba & Pagode',
  'Découvertes',
  'MPB',
  'AMPLIFIKA',
  'Dance/Électro',
  'Christian & Gospel',
  'Rock',
  'Indie',
  'RADAR',
  'GLOW',
  'EQUAL',
  'Ambiance',
  'Sport',
  'Détente',
  'Soirée',
  'À la maison',
  'Dormir',
  'Amour',
  'Cuisine et restauration',
  'Bien-être',
  'Focus',
  'Tendances',
  'Décennies',
  'Spotify\xa0Singles',
  'Villes',
  'Voyage',
  'Nature et bruits',
  'Latino',
  'RnB',
  'K-pop',
  'Auteurs-\u200bcompositeurs',
  'Music\xa0+\xa0Talk',
  'Fresh\xa0Finds',
  'Enfants et famille',
  'TV et cinéma',
  'Gaming',
  'Animés',
  'Influenceurs',
  'Netflix',
  'Étudiants'],
 'ids': ['0JQ5DAt0tbjZptfcdMSKl3',
  '0JQ5DAqbMKFz6FAsUtgAab',
  '0JQ5DAqbMKFNN285FvRwOf',
  '0JQ5DAqbMKFKGQWi5egawY',
  '0JQ5DAqbMKFJCTDO6ndZMP',
  '0JQ5DAqbMKFEC4WFtoNRpw',
  '0JQ5DA

In [79]:
filter_categories = ["Pop","Sertanejo","Hip-Hop","Dance/Électro", "Rock","Latino", "RnB"]

In [80]:

categories_df = pd.DataFrame(categories)
categories_df = categories_df[categories_df['names'].isin(filter_categories)]
categories_df = categories_df.reset_index(drop=True)
categories_df

,names,ids
0,Sertanejo,0JQ5DAqbMKFJCTDO6ndZMP
1,Pop,0JQ5DAqbMKFEC4WFtoNRpw
2,Hip-Hop,0JQ5DAqbMKFQ00XGBls6ym
3,Dance/Électro,0JQ5DAqbMKFHOzuVTgTizF
4,Rock,0JQ5DAqbMKFDXXwE9BDJAr
5,Latino,0JQ5DAqbMKFxXaXKP7zcDp
6,RnB,0JQ5DAqbMKFEZPnFQSFB1T


### Buscando playlist


In [81]:
playlists = {
    "ids": [],
    "messages": [],
}

# Obter playlists e mensagens para cada categoria
playlists_data = [get_category_playlists(item, limit=50, offset=0) for item in categories_df['ids']]

# Extrair ids e mensagens
for track_collection in playlists_data:
    items = track_collection['playlists']['items']
    message = track_collection['message']
    for item in items:
        playlists["ids"].append(item['id'])
        playlists["messages"].append(message)



In [82]:
playlists_df=pd.DataFrame(playlists)
playlists_df

,ids,messages
0,37i9dQZF1DXdSjVZQzv2tl,Sertanejo
1,37i9dQZF1DX5lAV5ZGu9ik,Sertanejo
2,37i9dQZF1DXc5EXfkDXlmk,Sertanejo
3,37i9dQZF1DX3R59Eoqqsy8,Sertanejo
4,37i9dQZF1DXcC14V9ZdJ9I,Sertanejo
...,...,...
267,37i9dQZF1DWYmmr74INQlb,R&B
268,37i9dQZF1DWSfMe9z89s9B,R&B
269,37i9dQZF1DX4y8h9WqDPAE,R&B
270,37i9dQZF1DWXnexX7CktaI,R&B


In [83]:
playlists_ids = playlists_df['ids'].tolist()
len(playlists_ids)

272

## Spotpy


In [84]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import re

In [85]:
# leaving these public because there's no need to make env-vars
cid = "e19f868c40394ceeb90e778daf20f9c7"
secret = "6be8e17dfbe347f0976fbdf00f500bc9"

In [209]:
# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [88]:
import concurrent.futures

def get_track_uris(playlist_id):
    tracks = sp.playlist_tracks(playlist_id)["items"]
    return [x["track"]["uri"] for x in tracks if x["track"]]

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(get_track_uris, playlists_ids))


In [129]:
playlists_df

,ids,messages
0,37i9dQZF1DXdSjVZQzv2tl,Sertanejo
1,37i9dQZF1DX5lAV5ZGu9ik,Sertanejo
2,37i9dQZF1DXc5EXfkDXlmk,Sertanejo
3,37i9dQZF1DX3R59Eoqqsy8,Sertanejo
4,37i9dQZF1DXcC14V9ZdJ9I,Sertanejo
...,...,...
267,37i9dQZF1DWYmmr74INQlb,R&B
268,37i9dQZF1DWSfMe9z89s9B,R&B
269,37i9dQZF1DX4y8h9WqDPAE,R&B
270,37i9dQZF1DWXnexX7CktaI,R&B


In [177]:
track_ids_df=pd.DataFrame(results)
#remove spotify:track:
track_ids_df = track_ids_df.replace(to_replace=r'spotify:track:', value='', regex=True)

In [178]:
#add messages to the dataframe in the first column
track_ids_df.insert(0, "message", playlists_df['messages'])
track_ids_df

,message,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,Sertanejo,5rHhO6SBSzo3E9m3KHyCtt,7pkpWcko6KEZANj0xKsCsu,1wubETHu3SeJHfbz9itEH0,7l9Tq5phzkLnhWUyfp702G,7f1tZNtc1t456ybBsl5sfK,3EJfFRgtbIYzBQdlc6PO9V,5sKGmZDmkGwFed5dPpZpZL,3JLhwgadVkN8zckxL1ICrI,0KDxhAvKeiVFL1bZccQVJs,...,None,None,None,None,None,None,None,None,None,None
1,Sertanejo,5EQQcQUoF1D2phHXV10Eq6,5rHhO6SBSzo3E9m3KHyCtt,7yD1kE5drOxjemNWdUrKNZ,08C7nkUFIk8BxtKxx8opXd,1wubETHu3SeJHfbz9itEH0,2Li8tWQBprZEjyDVQaSpTC,0BvzY9PCRufFPt51fvnhOI,6o5CBptHCsb4pyQV9l3TBU,7FdGeazienHpcYz9KYAuWW,...,None,None,None,None,None,None,None,None,None,None
2,Sertanejo,53PpijFATt35Zxfl6LTxoK,3GmJxfnUDrIs1iCfKUELFz,4AAIAVjSyiq1N7SXKkwXZe,4qsOrxBv09HhNSpsgMRXdC,38MpAndIjGAyVdcC6npX95,3tc8Z4lGzOIwmIVftsxO6o,4YhPcVIxIAJrLhFVPGbJkE,4UANYptjv2kpV6IY7IjXoJ,4SjkLzyFBVXaxuB6SbJyJN,...,None,None,None,None,None,None,None,None,None,None
3,Sertanejo,2fUELMuWTWZ0uAKud5nX2l,0BbTQBwcBhT00GbAv5UQ2R,7nFsuyRlLkIQEuRRTFb6zB,3ZoMHf4MvXA16kINwIa3C1,6NioqVBfrtXcmzQUxHxqGf,2Li8tWQBprZEjyDVQaSpTC,3JLhwgadVkN8zckxL1ICrI,0cmW66v6mOdc2yK6ioeBT5,5EmlbSncQN8QYgFK84Hr6Y,...,None,None,None,None,None,None,None,None,None,None
4,Sertanejo,3tc8Z4lGzOIwmIVftsxO6o,51pxwIxDVCbFHW8oyJFQXD,3laDGpbCmdrGOhjrcYIBYB,2dmGj5Nf01FcR6N4HPzAbL,6lK3nCiigk34SGqEjpQoqO,2gl2F5Z6ReNVh0wcyPLCKA,4AAIAVjSyiq1N7SXKkwXZe,6qDvzHbstT1Tr45okT8eN4,7ug4WpEEENdaGKoi5ZPLA7,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,R&B,7wBThXx7BGZHJJ3aN3OPvv,49FYlytm3dAAraYgpoJZux,5Ssv6DaKrW0HczVqx6zXdl,3aw9iWUQ3VrPQltgwvN9Xu,18JosZY3HzD3lMy6iOOSAY,7k6IzwMGpxnRghE7YosnXT,0W4NhJhcqKCqEP2GIpDCDq,34ceTg8ChN5HjrqiIYCn9Q,4iuNZTcvT9diFySSzVsnVS,...,74jMwg2zsFMICzW3Wems6h,1X1KDnhTnb28tpJlgOV3lD,6cZPfzVtkicrA3CyjqYKhu,02QQXgeAghpU4smdM7l7We,6uSa5iCMwPr10Ftpz8w09b,4bnjq8zqtCnxTGxll5ezOO,0SgbP5ZZOsUiHlE8EfeHma,5ZelMPn7vupvsTToUTvcbF,3lqT89gv8p3PYwXtr3ODjv,0bHs3ly4Bv5BlzE3KrePfX
268,R&B,37H7aRa7TWO7c0yvRnPcGg,0ZDwhNFZnPx9EIPCcwGUHC,6aU3cf7zDyZlfdiFNYeRNQ,6h1dtfc7PrKnpjIUAtZFR9,4HMop4Re0iucehmF7mgV27,2uvE4L5ZsYKpv8hbK4TIOt,0nn1kMhCkhQqqZdnKL7Ext,06nHXSt8RPzvdiXNrz3Ocv,0YMe6PHRbeDcN7KJdCG0bW,...,13emOZj9lqqVJg4JJ7ABBi,1KUAR4EL6XLMwQKjnKhVRn,0EQt3hJTjy2dYuxZp3cqo9,4qUlp9rJtbi9gBZa2Ie3Xx,7fIHFXaxaFZ9TLxdZeYIRD,2mBNIt6P5TPuly5JdKUPzH,7y4voxIo9QJT9ZODjKEBLS,275KNpBTmpEriHiYRgLM8t,4SUFYzuSOqqVIwZJrkGD1W,3CarOVyNEOcK89v0o3elHk
269,R&B,2Oo9Heg2qZNGaZIz3shwUx,2oVVaVY0LkzwAYYcyzon6Z,3CyrVEkcaJdSh8cM8IcRdD,4oQ49ddwS8JEatZvzObZIy,4trOnPnDeFU0jU8qa5i9xX,2nYeyMeqYDiFSYYtl2BWD6,0276oT52BTy2zKrTQyYPQt,2ncHgB3JjvnY4YcoMo8tGl,1lKFDImYTHzIGPtiQQZSwu,...,4LycrPCWsqESQ08I3ghkrT,6hfujWCUcaL9EW8XXUAFHF,2tCGA7eGKeXXAKS1DCxx9k,1QWSfV2BlRyUkTiNxPT3Qv,6xs1Km7sZrtFiusDMYv7MW,3ol70ZGonP9HTEQSKhZYcY,6sPUQOYM3gPHt6V6tUOGPD,1tUYtX5SrRj9MJQgZ9u2W3,0MGwGJTDAf6FDS98lIxcy6,6bAhP5wDZy04YjQ1NTcyO5
270,R&B,4iZ4pt7kvcaH6Yo8UoZ4s2,3xKsf9qdS1CyvXSMEid6g8,3EaJDYHA0KnX88JvDhL9oa,0JXXNGljqupsJaZsgSbMZV,5Nt2FMqnKI2mlp7lztURNo,5IgjP7X4th6nMNDh4akUHb,3pXF1nA74528Edde4of9CC,7DfFc7a6Rwfi3YQMRbDMau,7zFXmv6vqI4qOt4yGf3jYZ,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
def preencher_e_limpar_ids(df):
    # Inicializa uma lista para armazenar as linhas preenchidas
    filled_rows = []
    
    # Itera por cada categoria única no DataFrame
    for categoria in df['message'].unique():
        # Seleciona todas as linhas que correspondem à categoria atual
        categoria_df = df[df['message'] == categoria]
        ids_acumulados = []
        ids_unicos = set()  # Usado para evitar duplicatas
        
        # Acumula IDs não None e únicos para a categoria
        for i in range(len(categoria_df)):
            for x in categoria_df.iloc[i, 1:]:
                if pd.notna(x) and x not in ids_unicos:
                    ids_acumulados.append(x)
                    ids_unicos.add(x)
        
        # Divide os IDs acumulados em blocos de até 100 IDs e cria uma linha para cada bloco
        for start in range(0, len(ids_acumulados), 100):
            filled_row = [categoria] + ids_acumulados[start:start + 100] + [None] * (100 - len(ids_acumulados[start:start + 100]))
            filled_rows.append(filled_row)

    # Define as colunas do DataFrame e reconstrói com a coluna 'message'
    df_filled = pd.DataFrame(filled_rows, columns=['message'] + list(range(100)))
    
    # Retorna o DataFrame final
    return df_filled

# Exemplo de uso
track_ids_df_test = pd.DataFrame({
    "message": ["Sertanejo", "Sertanejo", "Sertanejo", "Rock", "Rock"],
    0: ["a", "e", "j", "k", "11"],
    1: ["b", "f", None, "l", "12"],
    2: ["d", None, None, "m", "13"],
    3: [None, None, None, "n", None]
})

track_ids_df_filled_test = preencher_e_limpar_ids(track_ids_df_test)
track_ids_df_filled_test



,message,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,Sertanejo,a,b,d,e,f,j,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Rock,k,l,m,n,11,12,13,None,None,...,None,None,None,None,None,None,None,None,None,None


In [180]:
track_ids_df['message'].unique()

array(['Sertanejo', 'Pop', 'Hip-Hop', 'Dance/Electronic', 'Rock', 'Latin',
       'R&B'], dtype=object)

In [181]:
track_ids_df_filled = preencher_e_limpar_ids(track_ids_df)

In [182]:
track_ids_df_filled

,message,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,Sertanejo,5rHhO6SBSzo3E9m3KHyCtt,7pkpWcko6KEZANj0xKsCsu,1wubETHu3SeJHfbz9itEH0,7l9Tq5phzkLnhWUyfp702G,7f1tZNtc1t456ybBsl5sfK,3EJfFRgtbIYzBQdlc6PO9V,5sKGmZDmkGwFed5dPpZpZL,3JLhwgadVkN8zckxL1ICrI,0KDxhAvKeiVFL1bZccQVJs,...,1R4bX3psbe5D2Wc03F3UvS,0bor3AJ8QV6RRbyV64ZsOO,368hYqtNCKCAeUduMoSHhh,0mVTQi9I4ehzSi9u9hLca5,3knAG9kd3jNNuJlZZUAOoQ,0KqVaOxvmYJMz10w3SbiaV,53PpijFATt35Zxfl6LTxoK,3GmJxfnUDrIs1iCfKUELFz,4AAIAVjSyiq1N7SXKkwXZe,4qsOrxBv09HhNSpsgMRXdC
1,Sertanejo,38MpAndIjGAyVdcC6npX95,3tc8Z4lGzOIwmIVftsxO6o,4YhPcVIxIAJrLhFVPGbJkE,4UANYptjv2kpV6IY7IjXoJ,4SjkLzyFBVXaxuB6SbJyJN,7McCAKExE8KsI2q4Y8wx4U,2Gvn46nwyQhUnQiFhlP7WX,3YvE7uo4ffseBoJPh8IhHp,7IIDB25xKfPbw595U53Ose,...,5IQ9Q55rxOVJSyptbzrSiQ,6CRiR9SG7apy1XDn8KtmjG,50PxMrpHMhcIHjevxG4MvN,0dPt1lYDrZ3cfT5HBNHjJ7,3mzbab4H0iZYkHggSEj0UX,43tMu6TIJB5H7Yo02biE7p,0sWWCKHy3Cxevn3yx8DKHt,71fueay4E0RK3X1xQsQdj9,6Emv8Lu3LVu2Ybn32Qlj6M,3d0KqfqP9ikJGvwnOYYX8z
2,Sertanejo,3gisEEogzZW8SPs0exvD4D,2CjfZFik0UQpymdog4vdjM,36K3zy7Wu60RchhJaMuYKN,4BmF1yc2kwGRcKXEaREwW6,3W8Oy4HpNaB3A2jQiH3CIG,7wKqfLqeAAD7dxgAgvDyZQ,095l0cDbBUgURjC4mTJxne,3QVl9YJKRyH331klQsfXRS,3GsEY43YrEzbFASVfDL8uJ,...,29lKPekKaTt7SpRMbIfFe6,0EMq5oTy960g88eIet1l1g,2SmxrZmtuPBQS7MRvssBTg,5GXjoulUhmrs8aveHZS2Hi,5uu61Dkp9jUdwDkn9wsxLc,7xtCgX2WPACexmWCLwsXlW,0Cksj6ceXYbLdXPHxdnzxR,5Q23x9RyABJ9vx09lNb9hs,2x2DLvYQZ0esvRw8Qyp06X,4TlOLjhHZjoVGF0qbQc996
3,Sertanejo,2P7cdwIoEMAl3qavXponC2,0He5PQBBA10GH88c73vqkQ,651qayNmTd9rPtIhqQkgXo,5AIcvmz8bKb9jAUeSMqoFj,256y8gclQRDVPLHXSD07Z9,7HYXb5RrUhhwxlnkhD0j6d,27HnfCdxjkA2y8e4flkvbg,26nqzMEy2Lz5tLR7cumNdH,2MEkfzmtPj7LUdXrz4hM8Y,...,7zmS0vPbWg2k6sfL3uqrGD,3PqU6gHWUaBTtIZi5oxEJ7,1mimc5QiUuwmVYdXkWvgOQ,7GScY8oE12cqTiNBzVQpUB,4vdFF0P7oTZoeKvnfIsbh3,579sk87MxQszhpGdqoHxnc,2slHCHYP7ogpGSsCumJT1a,0AIA4KsMsyqur8FbTVhGNT,1Z3QqD45wKKN6fUKBXSvcZ,64I2WXRrFWWxjRh8uTYYQB
4,Sertanejo,2nfnhFDhwMet9yZodpGfn3,0WrM1ON5poAsmKw3x5uPhD,6AENBD8I0mDGglFqrRZGu7,3zKJWk2l4uZ46KNRSFHriG,0AXfTPzrAMfHchWf2aOVfA,04xnlYSths6rviD2Agr0dZ,4Wl9xgqffYdxd0vZMm0FeO,1u6KujXHt5E1Uezt1R8mtb,3S8M3mZGIc10IdtmNTsrjQ,...,55vC6IXdrqntPOZeMQJ8Kd,1cmH3nQCYgR8lmq1ZFAXcI,39BkJtg9J6gy6IybhX6ClV,1YxF1GMnWW0aw38Fgz1jQl,1kjRfTPXU0rkHXhYR7usye,4neMZ5gXjse4vaOOEKkidj,5mGLuvOu3XY68bP3xRi1T9,35bwSFgKuQXofEOgHpvAeq,0YcZaFuGeIwAYtxpmdJNTy,0Fl543ELnqWqMP7WHI5Q0T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,R&B,35osWEtUUselhnoFpwA3X2,30c0CcK6RVr8tX8B19M3AW,7AvJrRBbe7U4eis5ifjf78,5m0X50W9ga5qghnXzeKNSe,1GkOOo0eZng6Q4zZ6rfAvj,6wSS1PXZcHbjIKHwQK6g1v,7o71W2N5PUPrzUDqigchCI,3qzHEwPUvoo5jX1a3htb0G,2kE7QJTNVVr3P0iYi7D0ia,...,3jVSV60WZE5jPE2a6KGNFy,5HM5trJxTouv3eUVcbW8sh,0J1YRLYhTG1MBosjidD7OI,7JKEA8xYDoFp4q0QBW2PGg,1AjQE5MtP5jHQMyDepbvLI,2SLZ2rX5773RS16mcqBxG7,2sITbbWIOeg2Lwp4WN2jqr,4dvQg9sD8k9y4qiEURuj8v,7tsRp7QKUBp6hc9bth0h7x,0eBAg5wJBu2Acv6o72iY4c
139,R&B,2J2hVpwVV25EK8Bvd3OgCK,5S2TCmPS3UUrG3ZK7NBa7b,5NaOWLOmVjcA3M5spRATN9,79XRUITVajhW5Snn7WF3YQ,7vAXxDFUHDPxWzSujxb8jB,6KS5iyM4uDuJ6oAacMSXha,5RgNhmyaWYtuIioRVoQzMR,13eNzMdXHUinoWsxcRXTCW,3OWcWJz1c0pIc6L3IDDhpP,...,4bnjq8zqtCnxTGxll5ezOO,0SgbP5ZZOsUiHlE8EfeHma,5ZelMPn7vupvsTToUTvcbF,3lqT89gv8p3PYwXtr3ODjv,0bHs3ly4Bv5BlzE3KrePfX,0ZDwhNFZnPx9EIPCcwGUHC,6aU3cf7zDyZlfdiFNYeRNQ,6h1dtfc7PrKnpjIUAtZFR9,4HMop4Re0iucehmF7mgV27,0nn1kMhCkhQqqZdnKL7Ext
140,R&B,06nHXSt8RPzvdiXNrz3Ocv,0YMe6PHRbeDcN7KJdCG0bW,0nmIWoirK2cbSD9duTDQJa,3CAws2VaEfaIvOeTcXVpGi,2bAVI1A5pTpfi6qloElMWa,6V9FSNT1V3LceWw9TjlXRg,13lHKa49RJRMGRNGftt5Ss,3ld1FzyNgpZwGn4h9cOXrq,4qZLrKsaGkYmVRGUviVPtk,...,3V8Cz6eVyTUCBWooVwNvR3,1upAmogfao9L253WeV9Jk5,6uL7XqegqsvKYRPUeRCPAR,5DLsTmQFQ3yCPowg1nreoh,6OiQNUMCLXJtJxOmUsoyFA,7eLiZCx55Yds7xwRiXOfRR,4a7Yuev0jhPYccwMpBu7kU,4MGD9rpRJlRj8jSfuFh4B9,0Jij0ywlT1cCn8MqmSmj4E,39QGmbxBiOxLRs76O9Wzy4
141,R&B,27UB48yRwGTsPsswfAbgN1,4iGY8xTqJnfTRaOKtyWweE,1vExubRVvilXeSg8S1IjBK,3gbTMPxtp57FiBtymz7Tqx,0iLEfYSsM1FZQRaqXdX0dq,6SsfouKNaxqKgxZbhgZd3v,6NlCxCDyNBtjwXw2sa9gkU,6V0bEMXp3ZynscGRT3QBiA,3lFbM9JdipWtVRzohsaOWk,...,4Oh1zkPShpImBJOBOKMM

In [188]:
len(track_ids_df_filled['message'].unique())==len(track_ids_df['message'].unique())

True

In [189]:
for i in range(len(track_ids_df_filled) - 1):
    if len(track_ids_df_filled.iloc[i].value_counts()) != 101:
        if track_ids_df_filled['message'][i] == track_ids_df_filled['message'][i + 1]:
            print("Erro na linha", i)

In [190]:
# Agrupa do 1 ao 100 em uma lista
track_ids_df_filled['track_ids'] = track_ids_df_filled.iloc[:, 1:].values.tolist()

# Remove as colunas que foram adicionadas na lista
track_ids_df_filled = track_ids_df_filled.iloc[:, [0, -1]]

track_ids_df_filled

,message,track_ids
0,Sertanejo,"[5rHhO6SBSzo3E9m3KHyCtt, 7pkpWcko6KEZANj0xKsCs..."
1,Sertanejo,"[38MpAndIjGAyVdcC6npX95, 3tc8Z4lGzOIwmIVftsxO6..."
2,Sertanejo,"[3gisEEogzZW8SPs0exvD4D, 2CjfZFik0UQpymdog4vdj..."
3,Sertanejo,"[2P7cdwIoEMAl3qavXponC2, 0He5PQBBA10GH88c73vqk..."
4,Sertanejo,"[2nfnhFDhwMet9yZodpGfn3, 0WrM1ON5poAsmKw3x5uPh..."
...,...,...
138,R&B,"[35osWEtUUselhnoFpwA3X2, 30c0CcK6RVr8tX8B19M3A..."
139,R&B,"[2J2hVpwVV25EK8Bvd3OgCK, 5S2TCmPS3UUrG3ZK7NBa7..."
140,R&B,"[06nHXSt8RPzvdiXNrz3Ocv, 0YMe6PHRbeDcN7KJdCG0b..."
141,R&B,"[27UB48yRwGTsPsswfAbgN1, 4iGY8xTqJnfTRaOKtyWwe..."


In [193]:
for i in range(len(track_ids_df_filled)):
    if len(track_ids_df_filled['track_ids'][i]) != 100:
        print("Erro na linha", i)

In [196]:
@cache
def fetch_track_features(track_uri):
    features = sp.audio_features(track_uri)[0]
    return features

In [206]:
track_ids_df_filled['track_ids'][0]
track_ids = track_ids_df_filled['track_ids'][0]
print("[" + ",".join(track_ids) + "]")

[5rHhO6SBSzo3E9m3KHyCtt,7pkpWcko6KEZANj0xKsCsu,1wubETHu3SeJHfbz9itEH0,7l9Tq5phzkLnhWUyfp702G,7f1tZNtc1t456ybBsl5sfK,3EJfFRgtbIYzBQdlc6PO9V,5sKGmZDmkGwFed5dPpZpZL,3JLhwgadVkN8zckxL1ICrI,0KDxhAvKeiVFL1bZccQVJs,3Na9gwLmFdEJH2l54AYwpk,2KdyhL41F9solmRiyLQ3yl,3Bkw2nrO4vXwql6BBDH5pq,6ukVCqH77f6cwRHyJBhGgy,2ThEetsRrVOEoCNpoFVDjM,7CutNxjzAKbs7EhBAlnRJt,0BvzY9PCRufFPt51fvnhOI,4dvHZesL5UKzfJnFH5bm3g,0ikHh6vrN2RcCANXwRnMHx,2WEhFQZBJqgY3MBTxK9r10,1NcqLEg2r2LXqMVbam2BJD,3whfTX0NBHhjynUlARMf7q,2lhTndwmmoMqMq8SSDfwbU,38MiIYlhGASln9jXnOmX04,1tAls12jSoUZPoj9cv7OAh,5qMyN3dzXKb5P1MiKyqupZ,0GbvYZsvz7bWyijBDESebX,1cIS13bqNfZcTs8LSK8r0a,6u9tu5yRDTHpWlY86J1GDa,6mTpK40yIRk1xGGNvRMHGg,1Ms6NNsLMqdC5v1qE3d89p,4A59ygOlOd3V0GbyEmUxKs,5QpcLTQjh0qO7OIuR9q8IR,0WiJIa3V9GdcJfcy1FhLjp,1WHDOr19zyyqu6XXIvdgfX,6X6jKoyw1OjogRghMEQAZV,5cdheFwwvj4mi3oJmTynKA,2DtHi3V76ntRw1RIKgYMfe,0pK0Vd1MCWy7AmsyFfIJb8,2ORC76autQ7QTfBCkZl4Wo,0HDMisLsuGNml98Xq7atpm,1VbunACv3Z4cGfBenC7oTx,18Ap6Yh17R9Qrs7LdOvkW2,6hNREzgHbVM5AkGwP4SPmU,0aGBJDgysb

In [210]:
sp.audio_features('5rHhO6SBSzo3E9m3KHyCtt')

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5rHhO6SBSzo3E9m3KHyCtt:
 Max Retries, reason: too many 429 error responses

In [201]:
track_features = {
    "danceability": [],
    "energy": [],
    "key": [],
    "loudness": [],
    "mode": [],
    "speechiness": [],
    "acousticness": [],
    "instrumentalness": [],
    "liveness": [],
    "valence": [],
    "tempo": [],
    "id": [],
    "uri": [],
    "track_href": [],
    "analysis_url": [],
    "duration_ms": [],
    "time_signature": [],
}
for i in range(len(track_ids_df_filled)):
    features = sp.audio_features(track_ids_df_filled["track_ids"][i])
    time.sleep(1)
    # print(features)
    # for features in audio_features:
    track_features["danceability"].append(features["danceability"])
    track_features["energy"].append(features["energy"])
    track_features["key"].append(features["key"])
    track_features["loudness"].append(features["loudness"])
    track_features["mode"].append(features["mode"])
    track_features["speechiness"].append(features["speechiness"])
    track_features["acousticness"].append(features["acousticness"])
    track_features["instrumentalness"].append(features["instrumentalness"])
    track_features["liveness"].append(features["liveness"])
    track_features["valence"].append(features["valence"])
    track_features["tempo"].append(features["tempo"])
    track_features["id"].append(features["id"])
    track_features["uri"].append(features["uri"])
    track_features["track_href"].append(features["track_href"])
    track_features["analysis_url"].append(features["analysis_url"])
    track_features["duration_ms"].append(features["duration_ms"])
    track_features["time_signature"].append(features["time_signature"])

print(track_features)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5rHhO6SBSzo3E9m3KHyCtt,7pkpWcko6KEZANj0xKsCsu,1wubETHu3SeJHfbz9itEH0,7l9Tq5phzkLnhWUyfp702G,7f1tZNtc1t456ybBsl5sfK,3EJfFRgtbIYzBQdlc6PO9V,5sKGmZDmkGwFed5dPpZpZL,3JLhwgadVkN8zckxL1ICrI,0KDxhAvKeiVFL1bZccQVJs,3Na9gwLmFdEJH2l54AYwpk,2KdyhL41F9solmRiyLQ3yl,3Bkw2nrO4vXwql6BBDH5pq,6ukVCqH77f6cwRHyJBhGgy,2ThEetsRrVOEoCNpoFVDjM,7CutNxjzAKbs7EhBAlnRJt,0BvzY9PCRufFPt51fvnhOI,4dvHZesL5UKzfJnFH5bm3g,0ikHh6vrN2RcCANXwRnMHx,2WEhFQZBJqgY3MBTxK9r10,1NcqLEg2r2LXqMVbam2BJD,3whfTX0NBHhjynUlARMf7q,2lhTndwmmoMqMq8SSDfwbU,38MiIYlhGASln9jXnOmX04,1tAls12jSoUZPoj9cv7OAh,5qMyN3dzXKb5P1MiKyqupZ,0GbvYZsvz7bWyijBDESebX,1cIS13bqNfZcTs8LSK8r0a,6u9tu5yRDTHpWlY86J1GDa,6mTpK40yIRk1xGGNvRMHGg,1Ms6NNsLMqdC5v1qE3d89p,4A59ygOlOd3V0GbyEmUxKs,5QpcLTQjh0qO7OIuR9q8IR,0WiJIa3V9GdcJfcy1FhLjp,1WHDOr19zyyqu6XXIvdgfX,6X6jKoyw1OjogRghMEQAZV,5cdheFwwvj4mi3oJmTynKA,2DtHi3V76ntRw1RIKgYMfe,0pK0Vd1MCWy7AmsyFfIJb8,2ORC76autQ7QTfBCkZl4Wo,0HDMisLsuGNml98Xq7atpm,1VbunACv3Z

In [97]:
# track_collection = {
#     "track_uri": [],
#     "track_name": [],
#     "artist_uri": [],
#     "artist_info": [],
#     "artist_name": [],
#     "artist_pop": [],
#     "artist_genres": [],
#     "album": [],
#     "track_pop": [],
# }
# playlists = list(playlists_df["ids"].unique())

# def lula(track_data, playlist_id):
#     for track in sp.playlist_tracks('37i9dQZF1DXdSjVZQzv2tl')["items"]:
#         # URI
#         track_uri = track["track"]["uri"]
#         track_data["track_uri"].append(track_uri)

#         # Track name
#         track_name = track["track"]["name"]
#         track_data["track_name"].append(track_name)

#         # Main Artist
#         artist_uri = track["track"]["artists"][0]["uri"]
#         track_data["artist_uri"].append(artist_uri)
#         artist_info = sp.artist(artist_uri)
#         track_data["artist_info"].append(artist_info)

#         # Name, popularity, genre
#         artist_name = track["track"]["artists"][0]["name"]
#         track_data["artist_name"].append(artist_name)

#         artist_pop = artist_info["popularity"]
#         track_data["artist_pop"].append(artist_pop)

#         artist_genres = artist_info["genres"]
#         track_data["artist_genres"].append(artist_genres)

#         # Album
#         album = track["track"]["album"]["name"]
#         track_data["album"].append(album)

#         # Popularity of the track
#         track_pop = track["track"]["popularity"]
#         track_data["track_pop"].append(track_pop)

# lula(track_collection, "")

In [194]:
track_ids

['4RmjlmdNRXdSZs9cYiqcBS',
 '1jAAv68G63cvQJKfwlOqtJ',
 '4nraNqonMyfc0U3o8VeXVu',
 '2YZ7CJV3Q1OGh7ZL9PYWcP',
 '1BJJbSX6muJVF2AK7uH1x4',
 '0b18g3G5spr4ZCkz7Y6Q0Q',
 '1jAAv68G63cvQJKfwlOqtJ',
 '4nraNqonMyfc0U3o8VeXVu',
 '4RmjlmdNRXdSZs9cYiqcBS',
 '3WOhcATHxK2SLNeP5W3v1v',
 '6RUKPb4LETWmmr3iAEQktW',
 '0qaWEvPkts34WF68r8Dzx9',
 '6GAvlyyIZM9uvNa5VPSFMK',
 '0Sebfib4HlLwEYR4vgd0Q3',
 '22hOKPKTAegLoLJKM33K4a',
 '4zonTpJqa47k7H1bixSynG',
 '46N3FCKFABRjNoNBVq4osr',
 '277aQ7uvOPlWmULkOdufco',
 '5L3kJPnAMEORynAJrBHMbx',
 '5eBLyfMZMbm6xa69h3LvPU',
 '65TcmTdJ6bqvYfC8UTsqiR',
 '2tHWfJzKknUb1bLCabCUp2',
 '7MkwoDv07H8iIXe4xYRsyr',
 '2SxnF7ibCTxMihwyiW7znz',
 '503OTo2dSqe7qk76rgsbep',
 '1FSvqHzMs4v8vb17Cwuf5U',
 '20oDX6ZLQmWIipsWKdADyw',
 '2PnlsTsOTLE5jnBnNe2K0A',
 '2C11qhXSCvVzGogwOTP6aZ',
 '42VsgItocQwOQC3XWZ8JNA',
 '2CCvvsFMwmrlUfDdPeIUox',
 '5MtBO8sJ42NiAQHP0gIsKK',
 '1eaovty3PjfGj6vmAWONPg',
 '0IVAlb0cFCH1cQ974iR1TU',
 '503OTo2dSqe7qk76rgsbep',
 '6lEIjrQNwJPecJ7mMXjhjo',
 '5g9lS8deSIxItFBmZRC4vN',
 

In [100]:
fetch_track_features(track_ids[0])

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=4RmjlmdNRXdSZs9cYiqcBS:
 Max Retries, reason: too many 429 error responses

In [ ]:
# track_features = []
# for track_id in track_ids:
#     features = fetch_track_features(track_id)
#     time.sleep(0.6)
#     track_features.append(features)


In [ ]:
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
client_id1 = os.getenv("CLIENT_ID1")
client_secret1 = os.getenv("CLIENT_SECRET1")
client_id2 = os.getenv("CLIENT_ID2")
client_secret2 = os.getenv("CLIENT_SECRET2")
client_id3 = os.getenv("CLIENT_ID3")
client_secret3 = os.getenv("CLIENT_SECRET3")
client_id4 = os.getenv("CLIENT_ID4")
client_secret4 = os.getenv("CLIENT_SECRET")


In [ ]:
@cache
def bolsolula(args):
    track_id, client = args
    features = client.audio_features([track_id])[0]
    time.sleep(1)
    return features

In [ ]:
df = pd.read_csv("df_bolsolula.csv")


In [ ]:
track_ids_with_features = df["id"].tolist()
track_ids_with_features

['4RmjlmdNRXdSZs9cYiqcBS',
 '1jAAv68G63cvQJKfwlOqtJ',
 '4nraNqonMyfc0U3o8VeXVu',
 '2YZ7CJV3Q1OGh7ZL9PYWcP',
 '1BJJbSX6muJVF2AK7uH1x4',
 '0b18g3G5spr4ZCkz7Y6Q0Q',
 '1jAAv68G63cvQJKfwlOqtJ',
 '4nraNqonMyfc0U3o8VeXVu',
 '4RmjlmdNRXdSZs9cYiqcBS',
 '3WOhcATHxK2SLNeP5W3v1v',
 '6RUKPb4LETWmmr3iAEQktW',
 '0qaWEvPkts34WF68r8Dzx9',
 '6GAvlyyIZM9uvNa5VPSFMK',
 '0Sebfib4HlLwEYR4vgd0Q3',
 '22hOKPKTAegLoLJKM33K4a',
 '4zonTpJqa47k7H1bixSynG',
 '46N3FCKFABRjNoNBVq4osr',
 '5L3kJPnAMEORynAJrBHMbx',
 '277aQ7uvOPlWmULkOdufco',
 '5eBLyfMZMbm6xa69h3LvPU',
 '2NKwz3gyK8da44pR2ZbiM1',
 '7MkwoDv07H8iIXe4xYRsyr',
 '65TcmTdJ6bqvYfC8UTsqiR',
 '2SxnF7ibCTxMihwyiW7znz',
 '4pwKy99a1IhAwjekz8Zf0c',
 '503OTo2dSqe7qk76rgsbep',
 '20oDX6ZLQmWIipsWKdADyw',
 '2PnlsTsOTLE5jnBnNe2K0A',
 '73yEnFvnQmSJNMi92fnz6m',
 '3tFed7YsjGnIfxeLEQwx3R',
 '2CCvvsFMwmrlUfDdPeIUox',
 '42VsgItocQwOQC3XWZ8JNA',
 '5MtBO8sJ42NiAQHP0gIsKK',
 '5Z01UMMf7V1o0MzF86s6WJ',
 '503OTo2dSqe7qk76rgsbep',
 '6lEIjrQNwJPecJ7mMXjhjo',
 '5g9lS8deSIxItFBmZRC4vN',
 

In [ ]:
old_track_ids = track_ids.copy() 

In [ ]:
pd.DataFrame(track_ids)

,0
0,2tHWfJzKknUb1bLCabCUp2
1,1FSvqHzMs4v8vb17Cwuf5U
2,2C11qhXSCvVzGogwOTP6aZ
3,1eaovty3PjfGj6vmAWONPg
4,1FSvqHzMs4v8vb17Cwuf5U
...,...
8010,7JYEuetCpLHJoeCNUgCnnw
8011,1oBOEvaT0kVI8X7GLeqrOn
8012,3FkuU53iVkl6rAXNOQSKJR
8013,0YTSDtaBMkwzkyxBuKmy6c


In [ ]:
for track_id in track_ids_with_features:
    if track_id in track_ids:
        track_ids.remove(track_id)

In [ ]:
len(track_ids)

8015

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import spotipy

# Lista de credenciais
credentials_list = [
    {'client_id':client_id, 'client_secret': client_secret},
    {'client_id': client_id1, 'client_secret': client_secret1},
    {'client_id': client_id2, 'client_secret': client_secret2},
    {'client_id': client_id3, 'client_secret': client_secret3},
    {'client_id': client_id4, 'client_secret': client_secret4},
]

# Cria clientes Spotify com credenciais diferentes
clients = [
    spotipy.Spotify(auth_manager=spotipy.oauth2.SpotifyClientCredentials(
        client_id=cred['client_id'],
        client_secret=cred['client_secret']
    )) for cred in credentials_list
]



# Associa cada track_id a um cliente, alternando entre eles
args_list = [(track_id, clients[i % len(clients)]) for i, track_id in enumerate(track_ids)]

with ThreadPoolExecutor(max_workers=5) as executor:
    try: 
        track_features = list(executor.map(bolsolula, args_list))
    except:
        pass

ModuleNotFoundError: No module named 'retry'

In [ ]:
track_features = []
for args in args_list:
    try:
        resp = bolsolula(args)
        track_features.append(resp)  
    except:
        break

Max Retries reached


In [ ]:
track_features

[]

In [ ]:
df_bolsolula = pd.concat([df, pd.DataFrame(track_features)])

In [ ]:
df_bolsolula.to_csv('df_bolsolula.csv', index=False)